## Extended Kalman Filter
I decided to implement this first in Python instead of just filling in the blanks and patching together the boilerplate code from Udacity.

This will allow me to code it up from scratch and really get an intimate understanding of Kalman Filters before I complete the implementation in C++.

Using a Jupyter notebook will also help me to take thorough notes on the math in the midst of the code.

In EKF.py, I recreate the normal Kalman Filter implementation that I already built in C++ during the lessons.  Then I filled in the gaps in the implementation.

In [1]:
from EKF import *

In [2]:
#//input file with laser and radar measurements
in_file_name_ = "../data/sample-laser-radar-measurement-data-1.txt"  

#//output file with the estimation (px, py, vx, vy), measurements (px and py meas) and ground truths (px, py, vx, vy)
out_file_name_ = "obj_pose-laser-radar-output.txt"

In [3]:
print "RMSE:",run_ekf(in_file_name_, (.163 , .000041))

RMSE: [[ 0.02448499]
 [ 0.06257636]
 [ 0.03133589]
 [ 0.01819227]]



# Discern an optimal set of noise parameters

In [4]:
import itertools
import glob

def meets_specifications(rmse):
    """
    Takes in both RMSE values
    Returns True if the RMSE is within specified values
    """
    first_rmse = rmse[0]
    second_rmse = rmse[1]
    if first_rmse[0] <= .07 and \
        first_rmse[1] <= .07 and \
        first_rmse[2] <= .6 and \
        first_rmse[3] <= .6 and \
        second_rmse[0] <= .2 and \
        second_rmse[1] <= .2 and \
        second_rmse[2] <= .5 and \
        second_rmse[3] <= .83:
        return True 

In [5]:
# Initialize a high value for the total mean squared error (Between both files)
best_total = 100
pro_noise_range = np.arange(0.161,.162,.001)
meas_noise_range = np.arange(0.0000405,0.0000415,.0000001)
i = 0
for pro_noise, meas_noise in itertools.product(pro_noise_range, meas_noise_range):
    i += 1
    total = 0
    rmse = [None,None]
    for j, in_file_name_ in enumerate(glob.glob("../data/*")):
        rmse[j] = run_ekf(in_file_name_, (pro_noise, meas_noise))
        total += np.sum(rmse[j])
    print str(i) + ":", pro_noise, meas_noise, total, "Best:", best_total
    if best_total > total and meets_specifications(rmse):
        best_total = total
        best_values = pro_noise, meas_noise, rmse

1: 0.161 4.05e-05 0.291394688643 Best: 100
2: 0.161 4.06e-05 0.284914450108 Best: 0.291394688643
3: 0.161 4.07e-05 0.285211716422 Best: 0.284914450108
4: 0.161 4.08e-05 0.289463715333 Best: 0.284914450108
5: 0.161 4.09e-05 0.295346788801 Best: 0.284914450108
6: 0.161 4.1e-05 0.302170153386 Best: 0.284914450108
7: 0.161 4.11e-05 0.309937937755 Best: 0.284914450108
8: 0.161 4.12e-05 0.318994857991 Best: 0.284914450108
9: 0.161 4.13e-05 0.329641641762 Best: 0.284914450108
10: 0.161 4.14e-05 0.341677946327 Best: 0.284914450108
11: 0.162 4.05e-05 0.322809922631 Best: 0.284914450108
12: 0.162 4.06e-05 0.309218263532 Best: 0.284914450108
13: 0.162 4.07e-05 0.296807595009 Best: 0.284914450108
14: 0.162 4.08e-05 0.287437468756 Best: 0.284914450108
15: 0.162 4.09e-05 0.284325156555 Best: 0.284914450108
16: 0.162 4.1e-05 0.286909345677 Best: 0.284325156555
17: 0.162 4.11e-05 0.292049871514 Best: 0.284325156555
18: 0.162 4.12e-05 0.298357329762 Best: 0.284325156555
19: 0.162 4.13e-05 0.30553805403

In [6]:
print best_values

(0.16200000000000001, 4.0900000000000012e-05, [matrix([[ 0.0245491 ],
        [ 0.06262924],
        [ 0.02555261],
        [ 0.01661092]]), matrix([[ 0.02132479],
        [ 0.01438511],
        [ 0.02233536],
        [ 0.09693804]])])


From the above, I figure that a good measurment noise is 0.0000409, and a good process noise is 0.162.

I found these value by finding optimal values for a wide range, then narrowing my each around optimal values further and searching again.  I did that iteration only a few times.

## Determine Noise Parameters for C++ Implementation
As it turns out, running the exact same algorithm in C++ produces very different results.
So I want to do the same process to optimize the noise parameters on the binary I built.
This is fun because I get to learn how to run bash from Python.

In [7]:
import subprocess

In [13]:
subprocess.call(["ls","../../","-la"])

0

Ok that doesn't work in Jupyter.  I'll use %bash

In [31]:
%%bash
../build/ExtendedKF \
../data/sample-laser-radar-measurement-data-1.txt \
../build/output.txt \
.0000409 .0000409 .162


EKF: 
0.049996
x_ =     8.46283
   0.243512
-0.00362542
 0.00199008
P_ =   2.51488e-07  2.36795e-318   1.00603e-05   1.6831e-318
  6.8558e-310   2.51488e-07 -1.35276e-310   1.00603e-05
  1.00603e-05  4.70699e-317   0.000402445  3.34564e-317
-1.35276e-310   1.00603e-05  7.17324e-310   0.000402445
0.050057
x_ =     8.46384
   0.243642
-0.00531998
 0.00187629
P_ =  5.95666e-07 -5.54011e-08  1.94755e-06 -1.10539e-06
-5.54011e-08   2.5176e-06 -1.10551e-06  4.02987e-05
 1.94755e-06 -1.10551e-06  3.94765e-05 -2.21274e-05
-1.10539e-06  4.02987e-05 -2.21274e-05  0.000807177
0.050057
x_ =     8.46332
   0.245591
-0.00835807
  0.0209565
P_ =  1.11179e-06 -1.77236e-07  1.36909e-05 -1.75911e-06
-1.77236e-07  7.26061e-06 -1.75923e-06  7.47235e-05
 1.36909e-05 -1.75923e-06  0.000440591 -1.74783e-05
-1.75911e-06  7.47235e-05 -1.74783e-05     0.001047
0.054951
x_ =    8.36741
  0.243861
  -1.77688
-0.0312882
P_ =  1.54996e-06 -5.04761e-07  2.20249e-06 -4.15578e-06
-5.04761e-07  1.88786e-05 -4.20256e-06

### It works!
Awesome!
Notice that the accuracy is very different:

    Accuracy - RMSE:
    0.0313691
    0.0319252
      0.37451
     0.422032
 
I hardcoded the values I discerned from my experiment above.  I'll have to figure out new values here.  
If that doesn't work, I'll just try something else to optimize.

So I can run it once from a cell in Jupyter.
But in order to do looping, I'd actually a have to run it from a Python script.
Or, I would have to write the loop in bash.
I don't want to rewrite the whole thing in bash.

# Subprocess actually does work in Jupyter
I just didn't know how to use it.

In [67]:
def run_cpp_ekf(file_name, noise_radar, noise_laser, noise_process):
    """
    Executes the binary for the EKF implementation
    and returns the RMSE for each file.
    """
    output =  subprocess.check_output(["../build/ExtendedKF",
                                       str(file_name),
                                       "../build/output.txt",
                                       str(noise_radar),
                                       str(noise_laser),
                                       str(noise_process)])


    rmse = output.splitlines()[-4:]
    return rmse

print run_cpp_ekf(in_file_name_, .0000409, .0000409, .162)

['0.183184', '0.193073', '0.566482', ' 1.54974']


# Niiiice 
Time to loop!

In [74]:
# Initialize a high value for the total mean squared error (Between both files)
best_total = 100
pro_noise_range = np.arange(0.161,.162,.001)
meas_noise_range = np.arange(0.0000405,0.0000415,.0000001)
i = 0
for pro_noise, noise_r, noise_l in itertools.product(pro_noise_range, meas_noise_range, meas_noise_range):
    i += 1
    total = 0
    rmse = [None,None]
    for j, in_file_name_ in enumerate(glob.glob("../data/*")):
        rmse[j] = run_cpp_ekf(in_file_name_, pro_noise, noise_r, noise_l)
        # Convert to floats
        rmse[j] = map(float, rmse[j])
        total += np.sum(rmse[j][0])
    print str(i) + ":", pro_noise, meas_noise, total, "Best:", best_total
    if best_total > total and meets_specifications(rmse):
        best_total = total
        best_values = pro_noise, meas_noise, rmse

1: 0.161 4.05e-05 1.624585 Best: 100
2: 0.161 4.05e-05 1.623789 Best: 100
3: 0.161 4.05e-05 1.622995 Best: 100
4: 0.161 4.05e-05 1.6222 Best: 100
5: 0.161 4.05e-05 1.621407 Best: 100
6: 0.161 4.05e-05 1.620613 Best: 100
7: 0.161 4.05e-05 1.619821 Best: 100
8: 0.161 4.05e-05 1.619028 Best: 100
9: 0.161 4.05e-05 1.618247 Best: 100
10: 0.161 4.05e-05 1.617455 Best: 100
11: 0.161 4.05e-05 1.62539 Best: 100
12: 0.161 4.05e-05 1.624584 Best: 100
13: 0.161 4.05e-05 1.62379 Best: 100
14: 0.161 4.05e-05 1.622995 Best: 100
15: 0.161 4.05e-05 1.622201 Best: 100
16: 0.161 4.05e-05 1.621408 Best: 100
17: 0.161 4.05e-05 1.620615 Best: 100
18: 0.161 4.05e-05 1.619832 Best: 100
19: 0.161 4.05e-05 1.61904 Best: 100
20: 0.161 4.05e-05 1.618259 Best: 100
21: 0.161 4.05e-05 1.626185 Best: 100
22: 0.161 4.05e-05 1.62538 Best: 100
23: 0.161 4.05e-05 1.624584 Best: 100
24: 0.161 4.05e-05 1.62379 Best: 100
25: 0.161 4.05e-05 1.622996 Best: 100
26: 0.161 4.05e-05 1.622202 Best: 100
27: 0.161 4.05e-05 1.621419 